In [ ]:
#create working directory

import warnings; warnings.filterwarnings("ignore");


import os#Miscellaneous operating system interfaces
os.chdir(r'C:\\Users\\rtreichl\\Documents\\competitions\\Santander')  #working directory

#%matplotlib inline allows graphics to show below each cell (or graphics in line)
# for some reason, %matplotlib inline won't work if comments are made in the same cell

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd #munging and wrangling
import numpy as np  #for arrays, etc.
import matplotlib.pyplot as plt #graphs/plots
import scipy.stats as stats

from itertools import combinations
from numpy import array,array_equal

#preprocessing
from sklearn import preprocessing

#Model Selection -> Metrics
from sklearn.metrics import roc_auc_score
from sklearn import cross_validation

#xgboost
import xgboost as xgb #xgboost

In [ ]:
#use dataframes for EDA
train = pd.read_csv('train.csv')
test  = pd.read_csv('test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
print(train.shape)
print(train.info())
print(test.shape)
print(test.info())

In [ ]:
#length of train
num_train = len(train)

In [ ]:
#combine train and test sets
df_all = pd.concat((train, test), axis=0, ignore_index=True)
print(df_all.shape)

In [ ]:
#flag values equal to 9999999999 or -999999 as missing

#replace as nan

col_list=list(test.columns[0:336])
for i in col_list:
    df_all[i+'_miss'] = np.where(np.logical_or(df_all[i] == 9999999999, df_all[i] ==-999999,),1.0,0.0)
    df_all[i]         = np.where(np.logical_or(df_all[i] == 9999999999,df_all[i] == -999999),np.nan,df_all[i])

print(df_all.shape)

In [ ]:
#some fields appear to also use -1 as a missing value....but not all
minus_one_list=['delta_imp_reemb_var17_1y3','delta_imp_trasp_var17_in_1y3','delta_imp_trasp_var33_in_1y3',
'delta_num_reemb_var17_1y3','delta_num_trasp_var17_in_1y3','delta_num_trasp_var33_in_1y3',]
for i in minus_one_list:
    df_all[i+'_miss'] = np.where(df_all[i] == -1,1.0,df_all[i+'_miss'])
    df_all[i]         = np.where(df_all[i] == -1,np.nan,df_all[i])
    
print(df_all.shape)

In [ ]:
# remove constant columns
remove = []
for col in df_all.columns:
    if df_all[col].std() == 0:
        remove.append(col)

df_all.drop(remove, axis=1, inplace=True)

print(df_all.shape)

In [ ]:
# remove duplicated columns

remove = []
c = df_all.columns
for i in range(len(c)-1):
    v = df_all[c[i]].values
    for j in range(i+1,len(c)):
        if np.array_equal(v,df_all[c[j]].values):
            remove.append(c[j])

df_all.drop(remove, axis=1, inplace=True)

print(df_all.shape)

In [ ]:
#impute missing values
#imputation strategy is most_frequent

#imp = preprocessing.Imputer(strategy='most_frequent',axis=1)
#imp = preprocessing.Imputer(strategy='median',axis=1)
imp = preprocessing.Imputer(strategy='mean',axis=1)


for i in list(df_all.columns[2:314]):
    temp=pd.DataFrame((imp.fit(df_all[i]).transform(df_all[i])).reshape(-1,1))
    df_all[i]=temp

In [ ]:
for i in list(train.columns[2:348]):
    stats.probplot(train[i], dist="norm", plot=plt)
    plt.title('QQPlot'+i)
    plt.show()

In [ ]:
#define train and test set from df_all
df_train = df_all.iloc[:num_train]
df_test = df_all.iloc[num_train:]
ID_test = df_test['ID']

In [ ]:
y_train=df_train['TARGET']

x_train = df_train.drop(['ID','TARGET'],axis=1).values

x_test = df_test.drop(['ID','TARGET'],axis=1).values

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
#linear model perform better when all variables are on the same scale
#linear models assume all multi-variate data is multivariant normal
#pca performs better when data are standardized.

#std_scale = preprocessing.StandardScaler().fit(x_train)

#x_train   = std_scale.transform(x_train)

#x_test    = std_scale.transform(x_test)


In [ ]:
#train['imp_ent_var16_ult1']=np.where(train['imp_ent_var16_ult1'] > 75000,75000,train['imp_ent_var16_ult1'])
#train['imp_op_var39_comer_ult1']=np.where(train['imp_op_var39_comer_ult1'] > 8000,8000,train['imp_op_var39_comer_ult1'])
#train['imp_op_var39_comer_ult3']=np.where(train['imp_op_var39_comer_ult3'] > 9000,9000,train['imp_op_var39_comer_ult3'])
#train['imp_op_var40_comer_ult1']=np.where(train['imp_op_var40_comer_ult1'] > 2600,2600,train['imp_op_var40_comer_ult1'])
#imp_op_var40_comer_ult3 6000
#imp_op_var40_efect_ult1  900

In [ ]:
#training and test sets for cross validation
x_traincv, x_testcv, y_traincv, y_testcv = cross_validation.train_test_split(x_train,y_train, test_size=0.50, random_state=0)

In [ ]:
#identify appropriate number of boost rounds

def xg_paraml(label,x_train,y_train,x_test,y_test):

    for bb in range(475,525,5):
        
        xgb_train_data = xgb.DMatrix(x_train, np.array(y_train))
        xgb_test_data=xgb.DMatrix(x_test)
        params={'objective':'binary:logistic','eval_metric' : 'auc','eta': 0.01 ,
                        'subsample':1.0, 'nthread' : -1, 
                #'max_depth':6, 'colsample_bytree':0.3
               }
        xgb_estimator = xgb.train(params, xgb_train_data, num_boost_round= bb)
        pred=xgb_estimator.predict(xgb_train_data)
        pred_test=xgb_estimator.predict(xgb_test_data)
        metric_tr=roc_auc_score(y_train, pred)
        metric_te=roc_auc_score(y_test, pred_test)
        print(label+ str(metric_tr) + '   '+
              str(metric_te) +
              ' boost rounds '+str(bb))

xg_paraml(label=' parameter ', x_train=x_traincv,y_train=y_traincv,x_test=x_testcv,y_test=y_testcv)

In [ ]:
#identify optimal parameters

def xg_paraml(label,x_train,y_train,x_test,y_test):
    cb_list=[0.51,0.52,0.53,0.54,0.55,0.56,0.57,0.58,0.59,0.60,0.61,0.62,0.63, 0.64,0.65,0.66,0.67,0.68,0.69]
    
    for md in range(5,30,1):
        for cb in cb_list:
                    xgb_train_data = xgb.DMatrix(x_train, np.array(y_train ))
                    xgb_test_data=xgb.DMatrix(x_test)
                    params={'objective':'binary:logistic','eval_metric' : 'auc','eta': 0.01 ,
                        'subsample':1.0, 'max_depth':md, 'colsample_bytree':cb,'nthreads':-1}
                    xgb_estimator = xgb.train(params, xgb_train_data, num_boost_round= 515)
                    pred=xgb_estimator.predict(xgb_train_data)
                    pred_test=xgb_estimator.predict(xgb_test_data)
                    metric_tr=roc_auc_score(y_train, pred)
                    metric_te=roc_auc_score(y_test, pred_test)
                    print(label+ str(metric_tr) + '   '+str(metric_te) +#' subsample '+str(ss)+
                          ' max_depth '+
                      str(md)+' colsample '+str(cb))


xg_paraml(label=' parameter ', x_train=x_traincv,y_train=y_traincv,x_test=x_testcv,y_test=y_testcv)

In [ ]:
#final model with submission file
#def xg_sub(label,x_train,y_train,x_test,parameters):
    #xgb parameters
    
params = {'objective':'binary:logistic','eval_metric' : 'auc', 'eta': 0.01,
         'subsample':1.0, 'max_depth':5, 'colsample_bytree':0.6,
          'nthread':-1}


    #train input matrix
xgb_train_data = xgb.DMatrix(x_train, y_train)
xgb_test_data  = xgb.DMatrix(x_test)

xgb_estimator = xgb.train(params, xgb_train_data, num_boost_round= 515)


train_pred=pd.DataFrame(xgb_estimator.predict(xgb_train_data))
test_pred =xgb_estimator.predict(xgb_test_data)

submission_xg = pd.DataFrame(columns=['ID','TARGET'])
submission_xg['ID']=ID_test
submission_xg['TARGET']=test_pred
submission_xg.to_csv('santander_515_1_5_06_mean.csv',index=False)